In [ ]:
import os
import sys

from pathlib import Path
import geopandas as gpd
import rioxarray as rxr
import numpy as np
import json

from src.io_vector import FileReader

from IPython.display import JSON

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib.patches import Patch

In [ ]:
aoi_fname = "area_intersects_border.geojson"
arg_border_fname = "admin_border_ARG.geojson"
results_fname = "classified_land.tif"
stats_fname = "stats.json"

In [ ]:
INPUT_DIR = Path(os.getenv("INPUT_DIR"))
OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR"))
RESULTS_DIR = OUTPUT_DIR.joinpath("results")
INTERMEDIATE_RESULTS_DIR = OUTPUT_DIR.joinpath("intermediate_results")

In [ ]:
aoi_path = INPUT_DIR.joinpath(aoi_fname)
argentina_borders_path = INPUT_DIR.joinpath(arg_border_fname)
results_path = RESULTS_DIR.joinpath(results_fname)
stats_path = RESULTS_DIR.joinpath(stats_fname)

classified_cover = INTERMEDIATE_RESULTS_DIR.joinpath("land_cover_intersect_reclassified.tif")
classified_slope = INTERMEDIATE_RESULTS_DIR.joinpath("slope_reclassified.tif")
classified_hand = INTERMEDIATE_RESULTS_DIR.joinpath("hand_processed_reclassified.tif")

# Load required files

In [ ]:
aoi_area = gpd.read_file(aoi_path)

In [ ]:
argentina_borders = gpd.read_file(argentina_borders_path)

In [ ]:
result = rxr.open_rasterio(results_path, masked=True).squeeze()
result = result.where(result != -9999, np.nan)

In [ ]:
with open(stats_path) as f:
    stats = json.load(f)

# Reproject if needed

In [ ]:
result_crs = result.rio.crs
aoi_area.to_crs(result_crs, inplace=True)
argentina_borders.to_crs(result_crs, inplace=True)

# Plot results

In [ ]:
# situational map
aoi_area.explore()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# plot the provinces
argentina_borders.plot(ax=ax, color='lightblue', edgecolor='black', linewidth=1, alpha=0.5)

# plot the aoi
aoi_area.plot(ax=ax, color='orange', edgecolor='black', linewidth=1)

# add titles and labels
ax.set_title('Area of interest')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# create legend
legend_elements = [Patch(facecolor='lightblue', edgecolor='black', label='Argentina provinces', alpha=0.5),
                   Patch(facecolor='orange', edgecolor='black', label='Area of interest')]

ax.legend(handles=legend_elements, loc='lower right')

plt.show()

In [ ]:
cmap = mcolors.ListedColormap(['lightgrey', 'orange', 'green'])
bounds = [1, 2, 3, 4]  #bounds for the colormap
norm = mcolors.BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(10, 10))
cbar = ax.imshow(result, cmap=cmap, norm=norm)

# colorbar with labels
cbar = fig.colorbar(cbar, ticks=[1.5, 2.5, 3.5])
cbar.ax.set_yticklabels(['Low suitability', 'Medium suitability', 'High suitability'])

# get the spatial resolution (we know the spatial resolution in x and y is uniform)
resolution_x, resolution_y = result.rio.resolution()

# scale bar length in meters
scale_bar_length = 5000

# scale bar length in data units (pixels)
scale_bar_length_in_units = scale_bar_length / resolution_x


# Add scale bar
fontprops = fm.FontProperties(size=12)
scalebar = AnchoredSizeBar(ax.transData,
                           scale_bar_length_in_units, f'{scale_bar_length / 1000:.1f} km', 'lower right',
                           pad=0.1,
                           color='black',
                           frameon=False,
                           size_vertical=1,
                           fontproperties=fontprops)
ax.add_artist(scalebar)

# Add north arrow
x, y, arrow_length = 0.1, 0.9, 0.1
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)

ax.set_title('Land Suitability')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')

plt.show()

# Display project statistics

In [ ]:
JSON(stats)